In [1]:
import sys, os
path_set = False
if path_set is False:
    sys.path.append('../')
    os.chdir('../')
    path_set = True

import constant
from datasets import data_loader as dl
from models.model_fectory import MDANet

import torch
import time
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import copy
import argparse
import shutil

from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

In [2]:
# setting path
test_case = 'test_alexnet_multi_1'
saved_model_path = os.path.join(constant.logs_root, test_case, 'best_model.pt')
submit_output = os.path.join(constant.logs_root, test_case, 'submit.txt')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# restoring model
mdan = MDANet(345, 3).to(device)
checkpoint = torch.load(saved_model_path)
mdan.load_state_dict(checkpoint['model_state_dict'])
mdan.eval()

MDANet(
  (feature_extractor): AlexNetExtractor(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9

In [6]:
test_dataset = dl.load_data('test', resize = (224,224))
test_dataloader = DataLoader(test_dataset, batch_size=1,shuffle=False, num_workers=8)

predictions = []

print('Predicting the test images...')
for inputs, img_path in test_dataloader:
    inputs = inputs.to(device)

    # forward
    with torch.set_grad_enabled(False):
        outputs = mdan.inference(inputs)
        _, preds = torch.max(outputs, 1)
        predictions.append((img_path, preds.item()))
        
result = []
for (image_path, pred) in predictions:
    result.append('{},{}\n'.format(image_path[0], pred))

Predicting the test images...


NameError: name 'test_case_place' is not defined

In [7]:
with open(submit_output, 'w+') as fout:
    fout.write('image_name,label\n')
    for line in result:
        fout.write(line)